In [56]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from ipywidgets import GridspecLayout
import joblib

In [26]:
def one_hot_encode_variables(df, categories_dict):
    for column, categories in categories_dict.items():
        if column in df.columns:
            for category in categories:
                dummy_column = f"{column}_{category}"
                df[dummy_column] = (df[column] == category).astype(int)
            df.drop(column, axis=1, inplace=True)
    return df

def binary_encode(df, columns_to_encode):
    mapping = {'Y': 1, 'N': 0}
    df[columns_to_encode] = df[columns_to_encode].map(mapping)
    return df

def ordinal_encode(df, columns_to_encode, custom_order):
    mapping = {val: idx for idx, val in enumerate(custom_order)}
    df[columns_to_encode] = df[columns_to_encode].map(mapping)
    return df

def preprocess_data_single_entry(data_entry, categorical_dict, binary_col, ordinal_col, custom_order):
    df = pd.DataFrame(data_entry)
    df = one_hot_encode_variables(df, categorical_dict)
    df = binary_encode(df, binary_col)
    df = ordinal_encode(df, ordinal_col, custom_order)
    df.fillna(df.median(), inplace=True)
    return df


In [59]:
def f(person_age, person_income, person_home_ownership, person_emp_length, loan_intent, loan_grade,
      loan_amnt, loan_int_rate, loan_percent_income, cb_person_default_on_file, cb_person_cred_hist_length):
   
   model = joblib.load('..\my_random_forest.joblib')
   data_entry = {
        'person_age': [person_age],
        'person_income': [person_income],
        'person_home_ownership': [person_home_ownership],
        'person_emp_length': [person_emp_length],
        'loan_intent': [loan_intent],
        'loan_grade': [loan_grade],
        'loan_amnt': [loan_amnt],
        'loan_int_rate': [loan_int_rate],
        'loan_percent_income': [loan_percent_income],
        'cb_person_default_on_file': [cb_person_default_on_file],
        'cb_person_cred_hist_length': [cb_person_cred_hist_length]
    }
   
   categorical_dict = {
    'person_home_ownership': ['MORTGAGE',  'OTHER', 'OWN',  'RENT'],
    'loan_intent': ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
   }

   binary_col = 'cb_person_default_on_file'
   ordinal_col = 'loan_grade'
   custom_order = ['A', 'B', 'C', 'D', 'E']
   customer_data_transformed = preprocess_data_single_entry(data_entry, categorical_dict, binary_col, ordinal_col, custom_order)
   
   prediction = model.predict(customer_data_transformed)
   predicted_probability = model.predict_proba(customer_data_transformed)[:,1]

   status = "granted" if prediction == 1 else "denied"
   return f"The credit has been {status} with probability {predicted_probability[0]}"



# Create a function to generate HTML for a heading with custom class
def generate_heading(text, css_class=''):
    return f"<h1 class='{css_class}'>{text}</h1>"

heading_widget = widgets.HTML(value=generate_heading("Credit Helper"))

# Create widgets for each criteria
person_age_widget = widgets.IntSlider(description="Person Age")
person_income_widget = widgets.FloatText(description="Person Income")
person_home_ownership_widget = widgets.Dropdown(options=['RENT', 'OWN', 'MORTGAGE', 'OTHER'], description="Home Ownership")
person_emp_length_widget = widgets.IntSlider(description="Employment Length (in years)")
loan_intent_widget = widgets.Dropdown(options=['EDUCATION', 'DEBTCONSOLIDATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE'], description="Loan Intent")
loan_grade_widget = widgets.Dropdown(options=['A', 'B', 'C', 'D', 'E'], description="Loan Grade")
loan_amnt_widget = widgets.FloatText(description="Loan Amount")
loan_int_rate_widget = widgets.FloatText(description="Interest Rate")
loan_percent_income_widget = widgets.FloatText(description="Percent Income")
cb_person_default_on_file_widget = widgets.Dropdown(options={'Yes': 'Y', 'No': 'N'}, description="Historical Default")
cb_person_cred_hist_length_widget = widgets.IntSlider(description="Credit History Length")


# Create a button to invoke the function
invoke_button = widgets.Button(description="Calculate")

def on_button_click(b):
   result_output.clear_output()
   with result_output:
       result = f(person_age_widget.value, person_income_widget.value, person_home_ownership_widget.value,
                  person_emp_length_widget.value, loan_intent_widget.value, loan_grade_widget.value,
                  loan_amnt_widget.value, loan_int_rate_widget.value,
                  loan_percent_income_widget.value, cb_person_default_on_file_widget.value,
                  cb_person_cred_hist_length_widget.value)
       print(result)
# Display widgets and their descriptions
display(heading_widget)   




#display(widgets.Label("Enter Criteria:"))
display(widgets.Label("Age"), person_age_widget)
display(widgets.Label("Annual Income"), person_income_widget)
display(widgets.Label("Home Ownership"), person_home_ownership_widget)
display(widgets.Label("Employment Length (in years)"), person_emp_length_widget)
display(widgets.Label("Loan Intent"), loan_intent_widget)
display(widgets.Label("Loan Grade"), loan_grade_widget)
display(widgets.Label("Loan Amount"), loan_amnt_widget)
display(widgets.Label("Interest Rate"), loan_int_rate_widget)
display(widgets.Label("Percent Income"), loan_percent_income_widget)
display(widgets.Label("Historical Default"), cb_person_default_on_file_widget)
display(widgets.Label("Credit History Length"), cb_person_cred_hist_length_widget)


# Display the invoke button and result output
display(invoke_button)
result_output = widgets.Output()
display(result_output)


Label(value='Age')

IntSlider(value=0, description='Person Age')

Label(value='Annual Income')

FloatText(value=0.0, description='Person Income')

Label(value='Home Ownership')

...

Dropdown(description='Historical Default', options={'Yes': 'Y', 'No': 'N'}, value='Y')

Label(value='Credit History Length')

IntSlider(value=0, description='Credit History Length')

Button(description='Calculate', style=ButtonStyle())

Output()